In [ ]:
# | default_exp classes.DomoPublish

In [ ]:
# |export
from fastcore.basics import patch_to

In [ ]:
# | exporti

from dataclasses import dataclass, field

from typing import Optional

import datetime as dt
import asyncio
import httpx

# import importlib
# import json
# import uuid
# import time

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.publish as publish_routes

# import Library.DomoClasses.DomoDataset as dmda
# import Library.DomoClasses.DomoLineage as dmdl

# Publish

In [ ]:
# | export
@dataclass
class DomoPublication_Subscription:
    subscription_id: str
    publication_id: str
    domain: str
    created_dt: Optional[dt.datetime] = None

    @classmethod
    def _from_json(cls, json):

        dd = json
        if not isinstance(dd, util_dd.DictDot):
            dd = util_dd.DictDot(json)

        return cls(
            subscription_id=dd.id,
            publication_id=dd.publicationId,
            domain=dd.domain,
            created_dt=dt.datetime.fromtimestamp(dd.created / 1000)
            if dd.created
            else None,
        )




In [ ]:
# | hide
import os
import pandas as pd

publication_id = "431005fd-ec85-41fe-88f6-7b888222aec2"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await publish_routes.get_publication_by_id(
    publication_id=publication_id, auth=token_auth
)

subscriber_ls = res.response.get("subscriptionAuthorizations")

[
    DomoPublication_Subscription._from_json(subscriber).__dict__
    for subscriber in subscriber_ls
]

[{'subscription_id': 'e1296172-c731-49d3-b543-b5c098cf5f8d',
  'publication_id': '431005fd-ec85-41fe-88f6-7b888222aec2',
  'domain': 'modocorp-admin.domo.com',
  'created_dt': datetime.datetime(2022, 10, 13, 15, 14, 51, 178000)}]

In [ ]:
# | export
@dataclass
class DomoPublication_Content:
    content_id: str
    entity_type: str
    entity_id: str
    entity_domain: str
    is_v2: bool
    is_direct_content: bool

    @classmethod
    def _from_json(cls, obj: dict):

        dd = obj
        if not isinstance(dd, util_dd.DictDot):
            dd = util_dd.DictDot(obj)

        dmpc = cls(
            content_id=dd.id,
            entity_type=dd.content.type,
            entity_id=dd.content.domoObjectId,
            entity_domain=dd.content.domain,
            is_v2=dd.isV2,
            is_direct_content=dd.useDirectContent,
        )

        return dmpc

    def to_api_json(self):
        temp_dict = {
            "domain": self.content_domain,
            "domoObjectId": self.entity_id,
            "customerId": self.content_domain,
            "type": self.entity_type,
        }
        return temp_dict


In [ ]:
# | hide
import os
import pandas as pd

publication_id = "431005fd-ec85-41fe-88f6-7b888222aec2"
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await publish_routes.get_publication_by_id(
    publication_id=publication_id, auth=token_auth
)

content_ls = res.response.get("children")

pd.DataFrame([DomoPublication_Content._from_json(content) for content in content_ls])

,content_id,entity_type,entity_id,entity_domain,is_v2,is_direct_content
0,ec909f98-b95b-4b04-b328-f21da172822f,PAGE,1880852133,domo-dojo.domo.com,True,True
1,460edf09-cca0-4a4c-bba0-51f82a1a5e84,DATASET,8848cdf1-28d4-46e9-93e4-7afb4b5ec1d2,domo-dojo.domo.com,True,True
2,aba70e14-ecf2-4c20-98af-8ffb4d56d183,DATASET,cf149711-df55-4dc3-a159-59a26d066baa,domo-dojo.domo.com,True,True
3,b70b4e6f-c4a9-444b-886a-774fa5405472,DATASET,ee24d234-303e-46aa-b062-9905dc0d8622,domo-dojo.domo.com,True,True
4,b9683bc2-33cc-4b10-8b31-9e394da85655,DATASET,ea859b06-5dbe-4c31-be76-de921447d550,domo-dojo.domo.com,True,True


In [ ]:
#| export
class DomoPublication_UnexpectedContentType(Exception):
    def __init__(self, publication_id, content_type, domo_instance):
        super().__init__(f"DomoPublication_Instantiation: Unexpected content type {content_type} in publication {publication_id} in {domo_instance}")

In [ ]:
# | export
@dataclass
class DomoPublication:
    id: str
    name: str
    description: str
    is_v2: bool
    created_dt: dt.datetime

    auth: dmda.DomoAuth = field(default=None, repr=False)

    subscription_authorizations: [DomoPublication_Subscription] = field(
        default_factory=list
    )
    content: [DomoPublication_Content] = field(default_factory=list)

    content_page_id_ls: [str] = field(default_factory=list)
    content_dataset_id_ls: [str] = field(default_factory=list)

    # lineage: dmdl.DomoLineage = None

    # def __post_init__(self):
    #     self.lineage = dmdl.DomoLineage(id=self.id,
    #                                     parent=self)

    @classmethod
    def _from_json(cls, obj, auth: dmda.DomoAuth = None):

        dd = util_dd.DictDot(obj)

        domo_pub = cls(
            id=dd.id,
            name=dd.name,
            description=dd.description,
            created_dt=dt.datetime.fromtimestamp(dd.created / 1000)
            if dd.created
            else None,
            is_v2=dd.isV2,
            auth=auth,
        )

        if dd.subscriptionAuthorizations and len(dd.subscriptionAuthorizations) > 0:
            domo_pub.subscription_authorizations = [
                DomoPublication_Subscription._from_json(sub)
                for sub in dd.subscriptionAuthorizations
            ]

        # publish only supports sharing pages and datasets
        if dd.children and len(dd.children) > 0:
            for child in dd.children:

                dmpc = DomoPublication_Content._from_json(child)
                domo_pub.content.append(dmpc)

                if dmpc.entity_type == "PAGE":
                    domo_pub.content_page_id_ls.append(dmpc.entity_id)

                elif dmpc.entity_type == "DATASET":
                    domo_pub.content_dataset_id_ls.append(dmpc.entity_id)

                else:
                    raise DomoPublication_UnexpectedContentType(
                        publication_id=domo_pub.id, 
                        content_type=dmpc.entity_type, 
                        domo_instance=auth.domo_instance)

        return domo_pub


In [ ]:
# | hide
import os
import pandas as pd

publication_id = "431005fd-ec85-41fe-88f6-7b888222aec2"
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await publish_routes.get_publication_by_id(
    publication_id=publication_id, auth=token_auth
)

DomoPublication._from_json(res.response).__dict__

{'id': '431005fd-ec85-41fe-88f6-7b888222aec2',
 'name': 'Variables Example from Domo-Dojo',
 'description': '',
 'is_v2': True,
 'created_dt': datetime.datetime(2022, 10, 13, 15, 14, 51, 178000),
 'auth': None,
 'subscription_authorizations': [DomoPublication_Subscription(subscription_id='e1296172-c731-49d3-b543-b5c098cf5f8d', publication_id='431005fd-ec85-41fe-88f6-7b888222aec2', domain='modocorp-admin.domo.com', created_dt=datetime.datetime(2022, 10, 13, 15, 14, 51, 178000))],
 'content': [DomoPublication_Content(content_id='ec909f98-b95b-4b04-b328-f21da172822f', entity_type='PAGE', entity_id='1880852133', entity_domain='domo-dojo.domo.com', is_v2=True, is_direct_content=True),
  DomoPublication_Content(content_id='460edf09-cca0-4a4c-bba0-51f82a1a5e84', entity_type='DATASET', entity_id='8848cdf1-28d4-46e9-93e4-7afb4b5ec1d2', entity_domain='domo-dojo.domo.com', is_v2=True, is_direct_content=True),
  DomoPublication_Content(content_id='aba70e14-ecf2-4c20-98af-8ffb4d56d183', entity_type

In [ ]:
@patch_to(DomoPublication, cls_method=True)
async def get_from_id(cls, publication_id=None, auth: dmda.DomoAuth = None):

    auth = auth or cls.auth

    publication_id = publication_id or cls.publication_id

    res = await publish_routes.get_publication_by_id(
        auth=auth, publication_id=publication_id
    )

    if not res.is_success:
        return None

    return cls._from_json(obj=res.response, auth=auth)

#### sample implementation of get_from_id


In [ ]:
import os
import pandas as pd

publication_id = "431005fd-ec85-41fe-88f6-7b888222aec2"
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

domo_publication = await DomoPublication.get_from_id(
    publication_id=publication_id, auth=token_auth
)
from pprint import pprint

pprint(domo_publication)

DomoPublication(id='431005fd-ec85-41fe-88f6-7b888222aec2',
                name='Variables Example from Domo-Dojo',
                description='',
                is_v2=True,
                created_dt=datetime.datetime(2022, 10, 13, 15, 14, 51, 178000),
                subscription_authorizations=[DomoPublication_Subscription(subscription_id='e1296172-c731-49d3-b543-b5c098cf5f8d',
                                                                          publication_id='431005fd-ec85-41fe-88f6-7b888222aec2',
                                                                          domain='modocorp-admin.domo.com',
                                                                          created_dt=datetime.datetime(2022, 10, 13, 15, 14, 51, 178000))],
                content=[DomoPublication_Content(content_id='ec909f98-b95b-4b04-b328-f21da172822f',
                                                 entity_type='PAGE',
                                                 entity_id='18808521

# Subscription

# DomoPublications

In [ ]:
@dataclass
class DomoPublications:

    @classmethod
    async def get_subscription_summaries(cls, 
                                         auth: dmda.DomoAuth,
                                         session: httpx.AsyncClient = None,
                                         return_raw: bool = False,
                                         debug_api: bool = False):
        """get instances subscription summaries"""

        res = await publish_routes.get_subscription_summaries(auth=auth,
                                                              session=session,
                                                              debug_api=debug_api
                                                              )

        if return_raw:
            return res

        if not res.is_success:
            return res

        sub_ls = res.response

        return [ sub for sub in sub_ls]
        
        


#### sample implementation of get_subscription_summaries

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await DomoPublications.get_subscription_summaries(auth = token_auth, return_raw= False)




[{'subscriptionId': 'a825f210-77cf-4313-a73d-4d2ae7cd3397',
  'publicationId': 'c7119a3e-f97d-4221-83c6-56ddd38073b0',
  'publisherDomain': 'datamavenllc-io.domo.com',
  'publicationSummary': {'id': 'c7119a3e-f97d-4221-83c6-56ddd38073b0',
   'name': 'New Feature Releases',
   'description': 'Domo Announcements',
   'dataSets': 4,
   'pages': 2,
   'cards': 42,
   'totalSubscribers': 0,
   'invalidSubscribers': 0,
   'invalidDatasetSubscribers': 0,
   'invalidPageSubscribers': 0,
   'invalidCardSubscribers': 0,
   'isV2': True,
   'hasDuplicate': False,
   'refreshing': False},
  'status': 'READY',
  'result': 'SUCCESS',
  'description': 'Success.',
  'datasets': 4,
  'pages': 2,
  'cards': 42,
  'created': 1674166687000,
  'updated': 1674166687000},
 {'subscriptionId': '3436f012-d9ff-4903-835e-3b88dcce11c9',
  'publicationId': 'd81b0a9a-e6f6-4ca0-b170-c57f91e75c22',
  'publisherDomain': 'datamavenllc-io.domo.com',
  'publicationSummary': {'id': 'd81b0a9a-e6f6-4ca0-b170-c57f91e75c22',
 

In [ ]:
#    def convert_content_to_dataframe(self, return_raw: bool = False):

#         output_ls = [{'plubication_id': self.id,
#                       'publication_name': self.name,
#                       'is_v2': self.is_v2,
#                       'publish_created_dt': self.created_dt,
#                       'entity_type': row.type,
#                       'entity_id': row.id
#                       } for row in self.content_entity_ls]

#         if return_raw:
#             return output_ls

#         return pd.DataFrame(output_ls)

#     def convert_lineage_to_dataframe(self, return_raw: bool = False):
#         import pandas as pd
#         import re

#         flat_lineage_ls = self.lineage._flatten_lineage()

#         output_ls = [{'plubication_id': self.id,
#                       'publication_name': self.name,
#                       'is_v2': self.is_v2,
#                       'publish_created_dt': self.created_dt,
#                       'entity_type': row.get('entity_type'),
#                       'entity_id': row.get('entity_id')
#                       } for row in flat_lineage_ls]

#         if return_raw:
#             return output_ls

#         return pd.DataFrame(output_ls)

#     @classmethod
#     async def create_publication(cls,
#                                  name: str,
#                                  content_ls: [DomoPublication_Content],
#                                  subscription_ls: [DomoPublication_Subscription],
#                                  unique_id: str = None,
#                                  description: str = None,
#                                  auth: dmda.DomoAuth = None,
#                                  debug: bool = False):

#         if not isinstance(subscription_ls, list):
#             subscription_ls = [subscription_ls]

#         auth = auth or cls.auth
#         domain_ls = []
#         content_json_ls = []
#         for sub in subscription_ls:
#             domain_ls.append(sub.domain)
#         for content_item in content_ls:
#             content_json_ls.append(content_item.to_json())

#         if not unique_id:
#             unique_id = str(uuid.uuid4())
#         if not description:
#             description = ''

#         body = publish_routes.generate_publish_body(url=f'{auth.domo_instance}.domo.com',
#                                                     sub_domain_ls=domain_ls,
#                                                     content_ls=content_json_ls,
#                                                     name=name,
#                                                     unique_id=unique_id,
#                                                     description=description,
#                                                     is_new=True)

#         res = await publish_routes.create_publish_job(auth=auth, body=body)
#         if debug:
#             print('Create the new Publish job')
#         if res.status != 200:
#             print(res)
#             await asyncio.sleep(2)
#             res = await publish_routes.get_publication_by_id(auth=auth, publication_id=unique_id)
#             if res.status != 200:
#                 return None
#             else:
#                 return cls._from_json(obj=res.response, auth=auth)

#         return cls._from_json(obj=res.response, auth=auth)

#     @classmethod
#     async def update_publication(cls,
#                                  name: str,
#                                  content_ls: [DomoPublication_Content],
#                                  subscription_ls: [DomoPublication_Subscription],
#                                  publication_id: str,
#                                  description: str = None,
#                                  auth: dmda.DomoAuth = None,
#                                  debug: bool = False):

#         if not isinstance(subscription_ls, list):
#             subscription_ls = [subscription_ls]

#         auth = auth or cls.auth
#         domain_ls = []
#         content_json_ls = []
#         for sub in subscription_ls:
#             domain_ls.append(sub.domain)
#         for content_item in content_ls:
#             content_json_ls.append(content_item.to_json())

#         if not description:
#             description = ''
#         body = publish_routes.generate_publish_body(url=f'{auth.domo_instance}.domo.com',
#                                                     sub_domain_ls=domain_ls,
#                                                     content_ls=content_json_ls,
#                                                     name=name,
#                                                     unique_id=publication_id,
#                                                     description=description,
#                                                     is_new=False)

#         res = await publish_routes.udpate_publish_job(auth= auth,
#                                                       publication_id=publication_id,
#                                                       body=body)
#         if debug:
#             print('Update Publish job by id')
#         if res.status != 200:
#             print(res)
#             await asyncio.sleep(2)
#             res = await publish_routes.get_publication_by_id(auth=auth, publication_id=publication_id)
#             if res.status != 200:
#                 return None
#             else:
#                 return cls._from_json(obj=res.response, auth=auth)

#         return cls._from_json(obj=res.response, auth=auth)

#     @classmethod
#     async def get_subscription_invites_list(cls, auth: dmda.DomoAuth,
#                                             debug: bool = False):

#         res = await publish_routes.get_subscription_invites_list(auth=auth,
#                                                                  debug=debug)
#         if debug:
#             print('Getting Publish subscription invites')

#         if res.status == 200:
#             return res.response

#     @classmethod
#     async def accept_invite_by_id(cls,
#                                   auth: dmda.DomoAuth,
#                                   subscription_id: str,
#                                   debug: bool = False):

#         res = await publish_routes.accept_invite_by_id(auth=auth,
#                                                        subscription_id=subscription_id,
#                                                        debug=debug)
#         if debug:
#             print(f'Accept invite by id {subscription_id}')

#         if res.status == 200:
#             return res.response


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 16)